# Week 2: Data Collection and Preprocessing
## Loading and Cleaning S-DoT Sensor Data

**Instructor**: Sohn Chul

---

## 🎯 Learning Objectives

By the end of this session, you will be able to:
1. Load and combine multiple S-DoT CSV files efficiently
2. Identify and handle missing values appropriately
3. Detect and remove outliers in sensor data
4. Perform data type conversions and datetime parsing
5. Create a clean, analysis-ready dataset

## 1. Setup and Import Libraries

In [ ]:
# Standard libraries
import os
import glob
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Datetime handling
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Progress bar for loops
from tqdm import tqdm

# Configure visualization
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Libraries loaded successfully")

## 2. Understanding S-DoT Data Structure

### 2.1 Data Characteristics

S-DoT sensor data has the following characteristics:
- **Temporal Resolution**: 10-minute intervals
- **File Organization**: Weekly CSV files (April-August 2025)
- **Expected Files**: 16 weekly files covering 5 months
- **Variables**: Temperature, Humidity, PM2.5, PM10, Noise
- **Encoding**: UTF-8 with Korean column names

In [ ]:
# Define data paths
BASE_PATH = '../..'  # Adjust based on your directory structure
SDOT_PATH = os.path.join(BASE_PATH, 's-dot')
OUTPUT_PATH = '../data/processed'

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_PATH, exist_ok=True)

print(f"📁 S-DoT data path: {SDOT_PATH}")
print(f"📁 Output path: {OUTPUT_PATH}")

## 3. Data Collection

### 3.1 List Available CSV Files

In [ ]:
# Find all CSV files in the S-DoT directory
csv_files = glob.glob(os.path.join(SDOT_PATH, '*.csv'))
csv_files.sort()  # Sort files chronologically

print(f"📊 Found {len(csv_files)} CSV files:")
print("="*50)

total_size = 0
for i, file in enumerate(csv_files, 1):
    file_size = os.path.getsize(file) / (1024 * 1024)  # Convert to MB
    total_size += file_size
    print(f"{i:2d}. {os.path.basename(file):40s} {file_size:8.2f} MB")

print("="*50)
print(f"Total size: {total_size:.2f} MB")

### 3.2 Load Sample File to Understand Structure

In [ ]:
# Load first file to understand structure
if csv_files:
    sample_file = csv_files[0]
    print(f"Loading sample: {os.path.basename(sample_file)}")
    
    # Read with Korean encoding
    df_sample = pd.read_csv(sample_file, encoding='utf-8', nrows=5)
    
    print("\n📋 Column names:")
    for i, col in enumerate(df_sample.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print("\n📊 Sample data:")
    display(df_sample.head())
    
    print("\n📈 Data types:")
    print(df_sample.dtypes)
else:
    print("No CSV files found. Creating sample data for demonstration...")
    
    # Create sample data structure
    df_sample = pd.DataFrame({
        '시리얼번호': ['S001'] * 5,
        '측정시간': pd.date_range('2025-04-01', periods=5, freq='10min'),
        '기온': np.random.normal(20, 5, 5),
        '습도': np.random.uniform(40, 80, 5),
        '미세먼지': np.random.exponential(25, 5),
        '초미세먼지': np.random.exponential(15, 5),
        '소음': np.random.normal(65, 10, 5)
    })

### 3.3 Define Column Mapping for Standardization

In [ ]:
# Define column name mapping (Korean to English)
COLUMN_MAPPING = {
    '시리얼번호': 'serial_number',
    '측정시간': 'datetime',
    '측정일시': 'datetime',  # Alternative name
    '기온': 'temperature',
    '온도': 'temperature',  # Alternative name
    '습도': 'humidity',
    '상대습도': 'humidity',  # Alternative name
    '미세먼지': 'pm10',
    'PM10': 'pm10',
    '초미세먼지': 'pm25',
    'PM2.5': 'pm25',
    '소음': 'noise',
    '소음도': 'noise'  # Alternative name
}

print("Column mapping defined:")
for korean, english in list(COLUMN_MAPPING.items())[:6]:
    print(f"  {korean:10s} → {english}")

## 4. Batch Data Loading

### 4.1 Function to Load and Standardize Single File

In [ ]:
def load_and_standardize_file(filepath, column_mapping=COLUMN_MAPPING):
    """
    Load a single CSV file and standardize column names.
    
    Parameters:
    -----------
    filepath : str
        Path to CSV file
    column_mapping : dict
        Dictionary mapping original to standardized column names
    
    Returns:
    --------
    pd.DataFrame
        Standardized dataframe
    """
    try:
        # Load CSV file
        df = pd.read_csv(filepath, encoding='utf-8')
        
        # Rename columns
        df = df.rename(columns=column_mapping)
        
        # Add source file information
        df['source_file'] = os.path.basename(filepath)
        
        return df
    
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None

# Test the function
if csv_files:
    test_df = load_and_standardize_file(csv_files[0])
    if test_df is not None:
        print(f"✅ Successfully loaded file with {len(test_df)} rows")
        print(f"Columns: {test_df.columns.tolist()}")

### 4.2 Load All Files with Progress Bar

In [ ]:
def load_all_sdot_files(file_list, column_mapping=COLUMN_MAPPING):
    """
    Load all S-DoT CSV files and combine into single dataframe.
    
    Parameters:
    -----------
    file_list : list
        List of file paths
    column_mapping : dict
        Column name mapping
    
    Returns:
    --------
    pd.DataFrame
        Combined dataframe
    """
    dataframes = []
    
    print(f"Loading {len(file_list)} files...")
    
    for filepath in tqdm(file_list, desc="Loading files"):
        df = load_and_standardize_file(filepath, column_mapping)
        if df is not None:
            dataframes.append(df)
    
    if dataframes:
        # Combine all dataframes
        combined_df = pd.concat(dataframes, ignore_index=True)
        print(f"\n✅ Successfully loaded {len(dataframes)} files")
        print(f"Total rows: {len(combined_df):,}")
        return combined_df
    else:
        print("❌ No data loaded")
        return None

# Load all data (or subset for testing)
# For testing, load only first 3 files
files_to_load = csv_files[:3] if len(csv_files) > 3 else csv_files

if files_to_load:
    df_all = load_all_sdot_files(files_to_load)
else:
    # Create synthetic data for demonstration
    print("Creating synthetic data for demonstration...")
    
    dates = pd.date_range('2025-04-01', '2025-04-07', freq='10min')
    n_sensors = 5
    
    df_all = pd.DataFrame({
        'serial_number': np.repeat([f'S{i:03d}' for i in range(1, n_sensors+1)], len(dates)),
        'datetime': np.tile(dates, n_sensors),
        'temperature': np.random.normal(22, 5, len(dates) * n_sensors),
        'humidity': np.random.uniform(40, 80, len(dates) * n_sensors),
        'pm10': np.random.exponential(40, len(dates) * n_sensors),
        'pm25': np.random.exponential(25, len(dates) * n_sensors),
        'noise': np.random.normal(65, 10, len(dates) * n_sensors),
        'source_file': 'synthetic_data.csv'
    })

## 5. Data Preprocessing

### 5.1 Data Type Conversion

In [ ]:
# Check current data types
print("Current data types:")
print(df_all.dtypes)
print("\n" + "="*50)

# Convert datetime column
if 'datetime' in df_all.columns:
    df_all['datetime'] = pd.to_datetime(df_all['datetime'])
    print("✅ Datetime column converted")

# Ensure numeric columns are float
numeric_columns = ['temperature', 'humidity', 'pm10', 'pm25', 'noise']
for col in numeric_columns:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors='coerce')
        print(f"✅ {col} converted to numeric")

print("\n" + "="*50)
print("Updated data types:")
print(df_all.dtypes)

### 5.2 Missing Value Analysis

In [ ]:
# Check for missing values
missing_counts = df_all.isnull().sum()
missing_percent = (missing_counts / len(df_all)) * 100

missing_summary = pd.DataFrame({
    'Missing_Count': missing_counts,
    'Missing_Percent': missing_percent
})

print("Missing Value Summary:")
print(missing_summary[missing_summary['Missing_Count'] > 0])

# Visualize missing patterns
if missing_counts.sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_summary[missing_summary['Missing_Count'] > 0]['Missing_Percent'].plot(kind='bar')
    plt.title('Missing Values by Column')
    plt.xlabel('Column')
    plt.ylabel('Missing Percentage (%)')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("\n✅ No missing values found!")

### 5.3 Handle Missing Values

In [ ]:
def handle_missing_values(df, method='interpolate'):
    """
    Handle missing values in the dataframe.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    method : str
        Method to handle missing values:
        - 'drop': Remove rows with missing values
        - 'forward_fill': Forward fill
        - 'interpolate': Linear interpolation
        - 'mean': Fill with mean value
    
    Returns:
    --------
    pd.DataFrame
        Dataframe with handled missing values
    """
    df_processed = df.copy()
    
    numeric_columns = df_processed.select_dtypes(include=[np.number]).columns
    
    if method == 'drop':
        df_processed = df_processed.dropna()
    
    elif method == 'forward_fill':
        df_processed[numeric_columns] = df_processed[numeric_columns].fillna(method='ffill')
    
    elif method == 'interpolate':
        # Group by sensor and interpolate
        if 'serial_number' in df_processed.columns:
            for col in numeric_columns:
                df_processed[col] = df_processed.groupby('serial_number')[col].transform(
                    lambda x: x.interpolate(method='linear', limit_direction='both')
                )
        else:
            df_processed[numeric_columns] = df_processed[numeric_columns].interpolate(method='linear')
    
    elif method == 'mean':
        for col in numeric_columns:
            df_processed[col].fillna(df_processed[col].mean(), inplace=True)
    
    return df_processed

# Apply missing value handling
df_clean = handle_missing_values(df_all, method='interpolate')

# Check results
print("Missing values after handling:")
print(df_clean.isnull().sum())
print(f"\n✅ Missing values handled using interpolation")
print(f"Original shape: {df_all.shape}")
print(f"Clean shape: {df_clean.shape}")

### 5.4 Outlier Detection and Removal

In [ ]:
def detect_outliers_iqr(df, columns, threshold=1.5):
    """
    Detect outliers using IQR method.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    columns : list
        Columns to check for outliers
    threshold : float
        IQR multiplier for outlier detection
    
    Returns:
    --------
    pd.DataFrame
        Boolean dataframe indicating outliers
    """
    outliers = pd.DataFrame(index=df.index)
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers[col] = (df[col] < lower_bound) | (df[col] > upper_bound)
            
            n_outliers = outliers[col].sum()
            percent = (n_outliers / len(df)) * 100
            
            print(f"{col:15s}: {n_outliers:6d} outliers ({percent:.2f}%)")
            print(f"  Range: [{lower_bound:.2f}, {upper_bound:.2f}]")
    
    return outliers

# Detect outliers
print("Outlier Detection Results:")
print("="*50)

numeric_cols = ['temperature', 'humidity', 'pm10', 'pm25', 'noise']
outliers = detect_outliers_iqr(df_clean, numeric_cols, threshold=3.0)

In [ ]:
# Visualize outliers
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    if col in df_clean.columns and i < len(axes):
        axes[i].boxplot(df_clean[col].dropna())
        axes[i].set_title(f'{col.capitalize()} Distribution')
        axes[i].set_ylabel('Value')
        axes[i].grid(True, alpha=0.3)

# Hide extra subplot
if len(numeric_cols) < len(axes):
    axes[-1].set_visible(False)

plt.suptitle('Outlier Detection using Boxplots', fontsize=16)
plt.tight_layout()
plt.show()

### 5.5 Apply Domain-Specific Constraints

In [ ]:
def apply_domain_constraints(df):
    """
    Apply domain-specific constraints to ensure data quality.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    
    Returns:
    --------
    pd.DataFrame
        Cleaned dataframe
    """
    df_constrained = df.copy()
    
    # Temperature: -30°C to 50°C (reasonable range for Seoul)
    if 'temperature' in df_constrained.columns:
        mask = (df_constrained['temperature'] >= -30) & (df_constrained['temperature'] <= 50)
        n_removed = (~mask).sum()
        df_constrained = df_constrained[mask]
        print(f"Temperature: Removed {n_removed} invalid values")
    
    # Humidity: 0% to 100%
    if 'humidity' in df_constrained.columns:
        mask = (df_constrained['humidity'] >= 0) & (df_constrained['humidity'] <= 100)
        n_removed = (~mask).sum()
        df_constrained = df_constrained[mask]
        print(f"Humidity: Removed {n_removed} invalid values")
    
    # PM2.5 and PM10: >= 0
    for col in ['pm25', 'pm10']:
        if col in df_constrained.columns:
            mask = df_constrained[col] >= 0
            n_removed = (~mask).sum()
            df_constrained = df_constrained[mask]
            print(f"{col.upper()}: Removed {n_removed} negative values")
    
    # Noise: 0 to 150 dB
    if 'noise' in df_constrained.columns:
        mask = (df_constrained['noise'] >= 0) & (df_constrained['noise'] <= 150)
        n_removed = (~mask).sum()
        df_constrained = df_constrained[mask]
        print(f"Noise: Removed {n_removed} invalid values")
    
    return df_constrained

# Apply constraints
print("Applying domain-specific constraints:")
print("="*50)
df_final = apply_domain_constraints(df_clean)

print("\n" + "="*50)
print(f"Final dataset shape: {df_final.shape}")
print(f"Data loss: {(1 - len(df_final)/len(df_clean))*100:.2f}%")

## 6. Data Quality Report

In [ ]:
def generate_quality_report(df):
    """
    Generate a comprehensive data quality report.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Cleaned dataframe
    
    Returns:
    --------
    dict
        Quality metrics
    """
    report = {}
    
    # Basic information
    report['total_records'] = len(df)
    report['date_range'] = f"{df['datetime'].min()} to {df['datetime'].max()}" if 'datetime' in df.columns else 'N/A'
    
    # Sensor information
    if 'serial_number' in df.columns:
        report['unique_sensors'] = df['serial_number'].nunique()
    
    # Data completeness
    report['completeness'] = (1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    
    # Statistical summary
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    report['statistics'] = df[numeric_cols].describe().to_dict()
    
    return report

# Generate report
quality_report = generate_quality_report(df_final)

print("📊 Data Quality Report")
print("="*50)
print(f"Total Records: {quality_report['total_records']:,}")
print(f"Date Range: {quality_report['date_range']}")
if 'unique_sensors' in quality_report:
    print(f"Unique Sensors: {quality_report['unique_sensors']}")
print(f"Data Completeness: {quality_report['completeness']:.2f}%")

print("\n📈 Statistical Summary:")
stats_df = pd.DataFrame(quality_report['statistics'])
print(stats_df.round(2))

## 7. Save Processed Data

In [ ]:
# Save cleaned data
output_file = os.path.join(OUTPUT_PATH, 'sdot_cleaned_data.csv')

try:
    df_final.to_csv(output_file, index=False, encoding='utf-8')
    file_size = os.path.getsize(output_file) / (1024 * 1024)  # MB
    print(f"✅ Data saved successfully!")
    print(f"📁 File: {output_file}")
    print(f"📊 Size: {file_size:.2f} MB")
    print(f"📈 Records: {len(df_final):,}")
except Exception as e:
    print(f"❌ Error saving file: {e}")

# Also save a smaller sample for quick testing
sample_file = os.path.join(OUTPUT_PATH, 'sdot_sample.csv')
df_final.head(10000).to_csv(sample_file, index=False, encoding='utf-8')
print(f"\n📁 Sample file saved: {sample_file}")

## 8. Assignment

### Week 2 Tasks:

1. **Data Loading** (25 points)
   - Load all 16 S-DoT CSV files for April-August 2025
   - Combine into a single dataframe
   - Document any loading issues encountered

2. **Data Cleaning** (30 points)
   - Handle missing values using appropriate methods
   - Identify and document outliers
   - Apply domain-specific constraints

3. **Data Analysis** (25 points)
   - Calculate the percentage of missing data per column
   - Create visualizations showing data distribution
   - Generate a data quality report

4. **Documentation** (20 points)
   - Document your preprocessing decisions
   - Explain why you chose specific methods
   - Create a summary of data characteristics

### Bonus Challenge (10 extra points):
- Implement a function to detect sensor malfunctions (e.g., constant values, sudden jumps)
- Create a visualization showing data availability over time for each sensor

### Submission:
- Complete notebook saved as `Week02_YourName.ipynb`
- Include your cleaned dataset (or a sample if too large)
- Submit via GitHub Pull Request

## 9. Summary

In this week, we covered:
- ✅ Loading multiple S-DoT CSV files
- ✅ Standardizing column names
- ✅ Handling missing values
- ✅ Detecting and removing outliers
- ✅ Applying domain constraints
- ✅ Generating data quality reports
- ✅ Saving clean datasets

### Key Takeaways:
1. **Data Quality is Critical**: Clean data is essential for accurate analysis
2. **Domain Knowledge Matters**: Understanding sensor limitations helps in preprocessing
3. **Documentation is Important**: Track all preprocessing decisions
4. **Efficiency Considerations**: Use batch processing for large datasets

### Next Week Preview:
**Week 3: Heat Index Calculation**
- Implement NOAA Heat Index formula
- Temperature unit conversions
- Calculate daily maximum heat index
- Identify heat wave events

---
**End of Week 2**

*Instructor: Sohn Chul*